In [12]:
%run train.py


In [14]:
# interrogate about the job
import requests
job_nb = 1
c = requests.get(f"http://localhost:8400/train?service=tls_park&job={job_nb}&timeout=10")
c.json()

{'head': {'job': 1, 'method': '/train'},
 'status': {'code': 404,
  'dd_code': 1003,
  'dd_msg': 'Job Not Found',
  'msg': 'NotFound'}}

In [15]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

s = Segmentation(
    'tls_park', 
    training_repo='/data1/sat/dataset_tls_park_444/train.txt',
    testing_repo='/data1/sat/dataset_tls_park_444/test.txt',
    port=8400,
    img_height=480,
    img_width=480,
    model_repo='/data1/sat/xolive/models/model_tls_park_debug',
    nclasses=2,
    template='deeplab_vgg16',
    iterations=15000,
    test_interval=500,
    snapshot_interval=1000,
    batch_size=5,
    test_batch_size=1,
    noise_prob=0.001,
    class_weights=[0.1, 1.0],
    model_postfix='test_park_xo3',
    weights='/data1/segmentation/vgg16_init_deeplab.caffemodel',
    tboard='/data1/tensorboard',
    gpuid=2
)#.run()
s

In [15]:
Classification('xo_bbc_5_classes', 
        training_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/train/',
        testing_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/test/',
        port=8400,
        model_repo='/data1/bbc_sounds/bbc_5_classes',
        layers='1CR32 1CR64 1CR128 1024',
        template='convnet',
               gpuid=1
        )#.run()
# TODO unchanged data

TypeError: __init__() got an unexpected keyword argument 'gpu_id'